# Output structure for GALAH DR3

## Author(s): Sven Buder (SB, WG4)

### History:
180926 SB Created

In [1]:
# Preamble for notebook 

# Compatibility with Python 3
from __future__ import (absolute_import, division, print_function)

try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Basic packages
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import sys
import collections
import glob
import pickle
import pandas

# Packages to work with FITS and (IDL) SME.out files
import astropy.io.fits as pyfits
import astropy.table as table
from astropy.table import Table
from scipy.io.idl import readsav

# Matplotlib and associated packages for plotting
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.transforms import Bbox,TransformedBbox
from matplotlib.image import BboxImage
from matplotlib.legend_handler import HandlerBase
from matplotlib._png import read_png
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import ListedColormap
import matplotlib.colors as colors

params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : True, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

_parula_data = [[0.2081, 0.1663, 0.5292], 
                [0.2116238095, 0.1897809524, 0.5776761905], 
                [0.212252381, 0.2137714286, 0.6269714286], 
                [0.2081, 0.2386, 0.6770857143], 
                [0.1959047619, 0.2644571429, 0.7279], 
                [0.1707285714, 0.2919380952, 0.779247619], 
                [0.1252714286, 0.3242428571, 0.8302714286], 
                [0.0591333333, 0.3598333333, 0.8683333333], 
                [0.0116952381, 0.3875095238, 0.8819571429], 
                [0.0059571429, 0.4086142857, 0.8828428571], 
                [0.0165142857, 0.4266, 0.8786333333], 
                [0.032852381, 0.4430428571, 0.8719571429], 
                [0.0498142857, 0.4585714286, 0.8640571429], 
                [0.0629333333, 0.4736904762, 0.8554380952], 
                [0.0722666667, 0.4886666667, 0.8467], 
                [0.0779428571, 0.5039857143, 0.8383714286], 
                [0.079347619, 0.5200238095, 0.8311809524], 
                [0.0749428571, 0.5375428571, 0.8262714286], 
                [0.0640571429, 0.5569857143, 0.8239571429], 
                [0.0487714286, 0.5772238095, 0.8228285714], 
                [0.0343428571, 0.5965809524, 0.819852381], 
                [0.0265, 0.6137, 0.8135], 
                [0.0238904762, 0.6286619048, 0.8037619048], 
                [0.0230904762, 0.6417857143, 0.7912666667], 
                [0.0227714286, 0.6534857143, 0.7767571429], 
                [0.0266619048, 0.6641952381, 0.7607190476], 
                [0.0383714286, 0.6742714286, 0.743552381], 
                [0.0589714286, 0.6837571429, 0.7253857143], 
                [0.0843, 0.6928333333, 0.7061666667], 
                [0.1132952381, 0.7015, 0.6858571429], 
                [0.1452714286, 0.7097571429, 0.6646285714], 
                [0.1801333333, 0.7176571429, 0.6424333333], 
                [0.2178285714, 0.7250428571, 0.6192619048], 
                [0.2586428571, 0.7317142857, 0.5954285714], 
                [0.3021714286, 0.7376047619, 0.5711857143], 
                [0.3481666667, 0.7424333333, 0.5472666667], 
                [0.3952571429, 0.7459, 0.5244428571], 
                [0.4420095238, 0.7480809524, 0.5033142857], 
                [0.4871238095, 0.7490619048, 0.4839761905], 
                [0.5300285714, 0.7491142857, 0.4661142857], 
                [0.5708571429, 0.7485190476, 0.4493904762],
                [0.609852381, 0.7473142857, 0.4336857143], 
                [0.6473, 0.7456, 0.4188], 
                [0.6834190476, 0.7434761905, 0.4044333333], 
                [0.7184095238, 0.7411333333, 0.3904761905], 
                [0.7524857143, 0.7384, 0.3768142857], 
                [0.7858428571, 0.7355666667, 0.3632714286], 
                [0.8185047619, 0.7327333333, 0.3497904762], 
                [0.8506571429, 0.7299, 0.3360285714], 
                [0.8824333333, 0.7274333333, 0.3217], 
                [0.9139333333, 0.7257857143, 0.3062761905], 
                [0.9449571429, 0.7261142857, 0.2886428571], 
                [0.9738952381, 0.7313952381, 0.266647619], 
                [0.9937714286, 0.7454571429, 0.240347619], 
                [0.9990428571, 0.7653142857, 0.2164142857], 
                [0.9955333333, 0.7860571429, 0.196652381], 
                [0.988, 0.8066, 0.1793666667], 
                [0.9788571429, 0.8271428571, 0.1633142857], 
                [0.9697, 0.8481380952, 0.147452381], 
                [0.9625857143, 0.8705142857, 0.1309], 
                [0.9588714286, 0.8949, 0.1132428571], 
                [0.9598238095, 0.9218333333, 0.0948380952], 
                [0.9661, 0.9514428571, 0.0755333333], 
                [0.9763, 0.9831, 0.0538]]

parula = ListedColormap(_parula_data, name='parula')
parula_zero = _parula_data[0]
parula_0 = ListedColormap(_parula_data, name='parula_0')
parula_0.set_bad((1,1,1))
parula_r = ListedColormap(_parula_data[::-1], name='parula_r')

willi_blau = [0.0722666667, 0.4886666667, 0.8467]

## Initialize GALAH DR3 output structure

In [2]:
release_data = collections.OrderedDict()

## Identifier

In [3]:
release_data['star_id'] = np.array(['JHHMMSSss+DDMMSSs'])
release_data['sobject_id'] = np.array([112233123401123])
release_data['source_id'] = np.array([1234567891234567890])
# release_data['ndfclass'] = np.array([123123123123])
# release_data['field_id'] = np.array([1234])

# WG4 information

In [4]:
release_data['wg4_field'] = np.array(['test'])
release_data['wg4_pipeline'] = np.array(['lbol'])
release_data['wg4_sp_flag'] = np.array([0])

In [5]:
solar_values = pyfits.getdata('../abundance_zeropoints/galahdr3_abundance_zeropoints.fits',1)

In [6]:
for each_solar_value in solar_values.dtype.names:
    release_data[each_solar_value] = solar_values[each_solar_value][0]
    if each_solar_value not in ['chi2_sp']:
        release_data['e_'+each_solar_value] = solar_values[each_solar_value][1]
        if each_solar_value in ['teff', 'logg', 'fe_h_atmo', 'vbroad', 'alpha_fe']:
            release_data['cov_e_'+each_solar_value] = 0.00
    if each_solar_value[0:2] == 'A_':
        release_data[each_solar_value[2:]+'_fe'] = 0.00
        release_data['e_'+each_solar_value[2:]+'_fe'] = 0.00
        
        release_data['flag_'+each_solar_value] = 0
        release_data['chi2_'+each_solar_value] = 0.00
        release_data['flux_'+each_solar_value] = 0.00

## 6D information

In [7]:
release_data['ra'] = np.array([0.00])
release_data['ra_error'] = np.array([0.00])

release_data['dec'] = np.array([0.00])
release_data['dec_error'] = np.array([0.00])

release_data['r_est'] = np.array([0.00])
release_data['r_lo'] = np.array([0.00])
release_data['r_hi'] = np.array([0.00])

release_data['r_hi'] = np.array([0.00])
release_data['r_hi'] = np.array([0.00])

release_data['r_hi'] = np.array([0.00])
release_data['r_hi'] = np.array([0.00])

release_data['rv_galah'] = np.array([0.00])
release_data['e_rv_galah'] = np.array([0.00])
release_data['c_rv_galah'] = np.array([0.00])
release_data['rv_gaia'] = np.array([0.00])
release_data['e_rv_gaia'] = np.array([0.00])

## WG3 information

In [8]:
release_data['red_flag'] = np.array([0])
release_data['ebv'] = np.array([0.00])
release_data['snr_c2_iraf'] = np.array([0.00])

release_data['flag_guess'] = np.array([0])
release_data['rv_guess'] = np.array([0.00])
release_data['e_rv_guess'] = np.array([0.00])
release_data['teff_guess'] = np.array([0.00])
release_data['logg_guess'] = np.array([0.00])
release_data['feh_guess'] = np.array([0.00])

## 2MASS information

In [9]:
release_data['j_m'] = np.array([0.00])
release_data['j_msigcom'] = np.array([0.00])
release_data['h_m'] = np.array([0.00])
release_data['h_msigcom'] = np.array([0.00])
release_data['ks_m'] = np.array([0.00])
release_data['ks_msigcom'] = np.array([0.00])
release_data['ph_qual_tmass'] = np.array(['AAA'])

## WISE and auxiliary information

In [10]:
release_data['w2mpro'] = np.array([0.00])
release_data['w2mpro_error'] = np.array([0.00])
release_data['ph_qual_wise'] = np.array(['AAAA'])
release_data['a_ks'] = np.array([0.00])
release_data['e_a_ks'] = np.array([0.00])

## Additional Gaia information

In [11]:
release_data['parallax'] = np.array([0.00])
release_data['parallax_error'] = np.array([0.00])
release_data['astrometric_excess_noise_sig'] = np.array([0.00])
release_data['astrometric_matched_observations'] = np.array([0])

release_data['phot_g_mean_mag'] = np.array([0.00])
release_data['bp_rp'] = np.array([0.00])

## Save GALAH DR3 output structure

In [12]:
release_pandas = pandas.DataFrame(release_data,columns=release_data.keys())
release_astropy = Table.from_pandas(release_pandas)
release_astropy.write('galah_dr3_output_structure.fits',overwrite=True)
release_pandas

,star_id,sobject_id,source_id,wg4_field,wg4_pipeline,wg4_sp_flag,teff,e_teff,cov_e_teff,logg,...,w2mpro_error,ph_qual_wise,a_ks,e_a_ks,parallax,parallax_error,astrometric_excess_noise_sig,astrometric_matched_observations,phot_g_mean_mag,bp_rp
0,JHHMMSSss+DDMMSSs,112233123401123,1234567891234567890,test,lbol,0,5779.0,104.0,0.0,4.42,...,0.0,AAAA,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
